In [5]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
import os

# Cargar las variables del archivo .env
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def obtener_score(mensaje, empresa, tamaño_empresa):
    prompt = f"""
Eres un asesor experto en marketing digital. Evalúa la calidad del siguiente lead que solicita servicios relacionados con páginas web, e-commerce o redes sociales.

Del 1 al 5, indica la probabilidad de que este lead quiera contratar un servicio (1 = interés bajo, 5 = interés alto).

Lead:
- Empresa: {empresa}
- Tamaño: {tamaño_empresa}
- Mensaje: "{mensaje}"

Solo responde con un número del 1 al 5, sin explicaciones.
"""

    try:
        print("🧠 Prompt:", prompt)  # DEBUG
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        print("📦 Respuesta OpenAI:", response)  # DEBUG
        return int(response.choices[0].message.content.strip())
    except Exception as e:
        print(f"❌ Error con lead: {mensaje}\n{e}")
        return None


In [6]:
def clasificar_necesidad(mensaje):
    mensaje = mensaje.lower()
    if "tienda" in mensaje or "ecommerce" in mensaje:
        return "E-commerce"
    elif "web" in mensaje or "página" in mensaje:
        return "Sitio Web"
    elif "instagram" in mensaje or "facebook" in mensaje or "redes" in mensaje:
        return "Redes Sociales"
    else:
        return "Otro"

df["necesidad"] = df["mensaje"].apply(clasificar_necesidad)


NameError: name 'df' is not defined

In [10]:
from openai import OpenAI

client = OpenAI()

def obtener_score(mensaje, empresa, tamaño_empresa):
    prompt = f"""
Eres un asesor experto en marketing digital. Evalúa la calidad del siguiente lead que solicita servicios relacionados con páginas web, e-commerce o redes sociales.

Del 1 al 5, indica la probabilidad de que este lead quiera contratar un servicio (1 = interés bajo, 5 = interés alto).


Lead:
- Empresa: {empresa}
- Tamaño: {tamaño_empresa}
- Mensaje: "{mensaje}"

Solo responde con un número del 1 al 5, sin explicaciones.
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return int(response.choices[0].message.content.strip())
    except Exception as e:
        print(f"❌ Error con lead: {mensaje}\n{e}")
        return None


In [11]:
df["lead_score"] = df.apply(
    lambda row: obtener_score(row["mensaje"], row["empresa"], row["tamaño_empresa"]),
    axis=1
)

df.to_csv("leads_con_scores.csv", index=False)
df


,nombre,email,empresa,tamaño_empresa,mensaje,lead_score,categoría,necesidad
0,Ana,ana@email.com,Tienda Ana,pequeña,Estoy interesada en vender productos sostenibles,4,🟢 Caliente,Otro
1,Luis,luis@email.com,Comercial Luis,mediana,Necesito urgentemente una solución de stock,3,🟢 Caliente,Otro
2,Sara,sara@email.com,Distribuciones Sara,grande,Busco un proveedor mayorista para mi ecommerce,4,🟢 Caliente,E-commerce


In [7]:
def categorizar(score):
    if score >= 4:
        return "🟢 Caliente"
    elif score == 3:
        return "🟡 Tibio"
    elif score <= 2:
        return "🔴 Frío"
    else:
        return "❓"

df["categoría"] = df["lead_score"].apply(categorizar)
df.to_csv("leads_con_scores.csv", index=False)
df


,nombre,email,empresa,tamaño_empresa,mensaje,lead_score,categoría
0,Ana,ana@email.com,Tienda Ana,pequeña,Estoy interesada en vender productos sostenibles,4,🟢 Caliente
1,Luis,luis@email.com,Comercial Luis,mediana,Necesito urgentemente una solución de stock,4,🟢 Caliente
2,Sara,sara@email.com,Distribuciones Sara,grande,Busco un proveedor mayorista para mi ecommerce,4,🟢 Caliente


In [1]:
df.to_excel("leads_con_scores.xlsx", index=False)


NameError: name 'df' is not defined